In [1]:
import pandas as pd
import os
import datetime
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from IPython.display import display

# Prepare data

In [2]:
class Data:
    def __init__(self, fileX, fileY):
        x = pd.read_csv(fileX, header=0, sep='\t')
        x = x.loc[:, ~x.columns.str.contains('^Unnamed')] #remove columns with undefined names containing values such as 'A', 'P' etc.
        x = x.loc[:, ~x.columns.str.contains('^Met_')] #remove samples with 15th tumor type (only present in test data)
        x = x.T #columns = genes (attributes); rows = samples
        self.x = x.drop(["Description", "Accession"], axis=0) # we don't need these columns
        
        print(self.x.shape)
        display(self.x.head())
        
        self.y = np.genfromtxt(fileY)       
        self.y = self.y[self.y != 14] #remove samples with 15th tumor type (only present in test data)
        
        print(self.y)

In [3]:
print("Train data:")
train = Data("GCM_Training.res", "GCM_Training.cls")
print("Test data:")
test = Data("GCM_Test.res", "GCM_Test.cls")

labels = pd.read_csv('Labels', header=None, sep=' ')
labels = labels.T[0]
print("Labels:")
print(labels)

Train data:
(144, 16063)


,0,1,2,3,4,5,6,7,8,9,...,16053,16054,16055,16056,16057,16058,16059,16060,16061,16062
Breast__Adeno_mBRT1_(8697),-73,-69,-48,13,-86,-147,-65,-71,-32,100,...,-134,352,-67,121,-5,-11,-21,-41,-967,-120
Breast__Adeno_mBRT2_(9078),-16,-63,-97,-42,-91,-164,-53,-77,-17,122,...,-51,244,-15,119,-32,4,-14,-28,-205,-31
Breast__Adeno_95_I_029,4,-45,-112,-25,-85,-127,56,-110,81,41,...,14,163,-14,7,15,-8,-104,-36,-245,34
Breast__Adeno_94_I_155,-31,-110,-20,-50,-115,-113,-17,-40,-17,80,...,26,625,18,59,-10,32,-2,10,-495,-37
Breast__Adeno_92_I_078,-33,-39,-45,14,-56,-106,73,-34,18,64,...,-69,398,38,215,-2,44,3,68,-293,-34


[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  2.  2.
  2.  2.  2.  2.  2.  2.  3.  3.  3.  3.  3.  3.  3.  3.  4.  4.  4.  4.
  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  5.  5.  5.  5.  5.  5.
  5.  5.  6.  6.  6.  6.  6.  6.  6.  6.  7.  7.  7.  7.  7.  7.  7.  7.
  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.
  8.  8.  8.  8.  8.  8.  9.  9.  9.  9.  9.  9.  9.  9. 10. 10. 10. 10.
 10. 10. 10. 10. 11. 11. 11. 11. 11. 11. 11. 11. 12. 12. 12. 12. 12. 12.
 12. 12. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13.]
Test data:
(46, 16063)


,0,1,2,3,4,5,6,7,8,9,...,16053,16054,16055,16056,16057,16058,16059,16060,16061,16062
Breast__Adeno_93_I_192,-44,-254,-166,-55,-130,-133,80,-62,1,22,...,-51,715,25,47,-270,-265,-290,-235,-826,-262
Breast__Adeno_09-B_005A,-13,-124,-206,-29,-117,-114,111,-20,-25,18,...,-57,695,10,57,-50,-45,-61,-22,-263,-52
Breast__Adeno_09-B_003A,-64.3,-184.9,-334.4,-102.2,-289,-145.2,-251.9,-150.9,-184,179.1,...,-173.4,1324.6,9.8,127.7,244,76.5,100.1,-69.6,-1444.3,-121.6
Prostate__Adeno_95_I_256,-22,-169,-164,-32,-49,-141,0,-22,16,67,...,21,170,20,35,-275,-230,-136,-228,-404,-259
Prostate__Adeno_P_0062,-28,-167,-158,100,-129,-232,148,-48,14,164,...,-66,1627,76,119,-6,68,-22,27,-1088,-58


[ 0.  0.  0.  1.  1.  2.  2.  2.  3.  3.  3.  4.  4.  4.  4.  4.  4.  5.
  5.  5.  6.  6.  7.  7.  8.  8.  8.  8.  8.  8.  9.  9.  9. 10. 10. 10.
 11. 11. 11. 12. 12. 12. 13. 13. 13. 13.]
Labels:
0            Breast
1          Prostate
2              Lung
3        Colorectal
4          Lymphoma
5           Bladder
6          Melanoma
7     Uterus__Adeno
8          Leukemia
9             Renal
10         Pancreas
11            Ovary
12     Mesothelioma
13              CNS
Name: 0, dtype: object


# Build the classifier

In [4]:
class MultiClassCancerClassifier:
    def __init__(self, classifiers):
        self.classifiers = classifiers
    
    def fit(self, data):
        print("Fitting state")
        for classifier in self.classifiers:
            classifier.fit(data)
        
    def predict(self, x):
        print("Predicting state")
        predictions = []
        for i in range(len(x)):
            #get margins for the 14 classifiers
            margins = []
            for classifier in self.classifiers:
                margins.append(classifier.predict(x.iloc[[i]])) #get margin for ith row
            # print("Margins: ", margins)
            #predict class with the biggest margin
            maxValue = max(margins, key=lambda prediction: prediction[2])
            predictions.append(maxValue[0])
            print("Best prediction is {}".format(maxValue))
            # print("All predictions {}".format(margins))
        return np.array(predictions)
    
    #helper function to print evaluation
    def evaluate(self, y_test, y_pred):
        # cm = confusion_matrix(y_test, y_pred)
        # print("Confusion matrix:\n", cm)
        accuracy = accuracy_score(y_test, y_pred )
        print("Accuracy:", accuracy)
        print(y_test)
        print(y_pred)
        return accuracy

In [5]:
# One Vs. All classifier
class OVA_SVM_Classifier:
    @staticmethod
    def createClassifiers(randomState, classesCount):
        result = []
        for i in range(classesCount):
            result.append(OVA_SVM_Classifier(i, randomState))
        return result
    
    def __init__(self, label, randomState):
        self.label = label # indicates what type of cancer is the positive target value
        self.classifier = SVC(kernel='linear', probability=True, random_state=randomState) # this kernel was used in the article
        
    def fit(self, data):
        print("Fitting classifier #{} {}".format(self.label, labels[self.label]))
        #compute positive and negative target values. 1 if equal to label, 0 otherwise.
        y_vals = []
        for val in data.y:
            if val == self.label:
                y_vals.append(1)
            else:
                y_vals.append(0)
        #fit the classifier
        self.classifier.fit(data.x, y_vals)
        
    def predict(self, x):
        probability = self.classifier.predict_proba(x)[0,1] #probability of being in class '1'
        return (self.label, 'ALL', probability)

In [6]:
# All Pairs classifier - classify between label pairs - label1 VS label2
class AP_SVM_Classifier_Inner:
    def __init__(self, label1, label2, randomState):
        # build classifier for predicting label1 instead of label2
        self.labels = [label1, label2]
        self.classifier = SVC(kernel='linear', probability=True, random_state=randomState)
    
    def fit(self, data):
        print("Fitting classifier #{} {}".format(self.labels, [labels[self.labels[0]], labels[self.labels[1]]]))
        dataIndex = (data.y == self.labels[0]) | (data.y == self.labels[1])
        x_vals = data.x[dataIndex]
        y_vals = []
        for val in data.y[dataIndex]:
            if val == self.labels[0]:
                y_vals.append(1)
            else:
                y_vals.append(0)   
        #fit the classifier     
        self.classifier.fit(x_vals, y_vals)
        
    def predict(self, x):
        probability = self.classifier.predict_proba(x)[0,1] #probability of being in class '1'
        return (self.labels[0], self.labels[1], probability)
    
class AP_SVM_Classifier:
    @staticmethod
    def createClassifiers(randomState, classesCount):
        result = []
        for i in range(classesCount):
            result.append(AP_SVM_Classifier(i, classesCount, randomState))
        return result
    
    def __init__(self, label, classesCount, randomState):
        self.labels = [label, 'ALL']
        self.classifiers = []
        for i in range(classesCount):
            if i == label:
                continue
            self.classifiers.append(AP_SVM_Classifier_Inner(label, i, randomState))
    
    def fit(self, data):
        for classifier in self.classifiers:
            classifier.fit(data)
            
    def predict(self, x):
        probability = 0
        for classifier in self.classifiers:
            value = classifier.predict(x)
            probability = probability + value[2]
        probability = probability / len(self.classifiers)
        return (self.labels[0], self.labels[1], probability)

# Make predictions and evaluate

In [7]:
def getPredictionTable(y, y_pred):
    table = pd.DataFrame()
    for classTarget in range(len(labels)):
        predictionsOk = (y == classTarget) & (y_pred == classTarget)
        predictionsErr = (y != classTarget) & (y_pred == classTarget)
        ok = sum(predictionsOk)
        err = sum(predictionsErr)
        table = table.append(pd.DataFrame({"Ok": [ok], "Err": [err]}, index = [labels[classTarget]]))
    return pd.DataFrame(table, columns=["Ok", "Err"])

def findMaxAccuracy(seed, iterations, classifiersCreator):
    np.random.seed(seed)
    accuracies = []
    for i in range(iterations):
        randomState = np.random.randint(100)
        print("\n\n{} Iteration {}: Using randomState = {}".format(datetime.datetime.now().time(), i, randomState))
        classifier = MultiClassCancerClassifier(classifiersCreator(randomState, len(labels)))
        classifier.fit(train)
        y_pred = classifier.predict(test.x)
        accuracy = classifier.evaluate(test.y, y_pred)
        accuracies.append((accuracy, randomState, getPredictionTable(test.y, y_pred)))
    for accuracy in accuracies:
        print("RandomState {}: Accuracy {}".format(accuracy[1], accuracy[0]))
        display(accuracy[2])

In [8]:
findMaxAccuracy(1, 5, OVA_SVM_Classifier.createClassifiers)



10:54:04.992856 Iteration 0: Using randomState = 37
Fitting state
Fitting classifier #0 Breast
Fitting classifier #1 Prostate
Fitting classifier #2 Lung
Fitting classifier #3 Colorectal
Fitting classifier #4 Lymphoma
Fitting classifier #5 Bladder
Fitting classifier #6 Melanoma
Fitting classifier #7 Uterus__Adeno
Fitting classifier #8 Leukemia
Fitting classifier #9 Renal
Fitting classifier #10 Pancreas
Fitting classifier #11 Ovary
Fitting classifier #12 Mesothelioma
Fitting classifier #13 CNS
Predicting state
Best prediction is (10, 'ALL', 0.2316031159824884)
Best prediction is (0, 'ALL', 0.6791984451724181)
Best prediction is (2, 'ALL', 0.2743603047249001)
Best prediction is (7, 'ALL', 0.8965620022371955)
Best prediction is (11, 'ALL', 0.11660106345340468)
Best prediction is (2, 'ALL', 0.5771902302805622)
Best prediction is (2, 'ALL', 0.9682154218558755)
Best prediction is (5, 'ALL', 0.1624759372243679)
Best prediction is (3, 'ALL', 0.6130425060612982)
Best prediction is (3, 'ALL', 0

Best prediction is (8, 'ALL', 0.9862530964240783)
Best prediction is (8, 'ALL', 0.996572452373937)
Best prediction is (9, 'ALL', 0.3569272601161667)
Best prediction is (9, 'ALL', 0.4948290069883952)
Best prediction is (9, 'ALL', 0.6986783343269706)
Best prediction is (10, 'ALL', 0.75013689691011)
Best prediction is (0, 'ALL', 0.28403826433415996)
Best prediction is (10, 'ALL', 0.0904301710894978)
Best prediction is (11, 'ALL', 0.4904824545519177)
Best prediction is (11, 'ALL', 0.525740320596182)
Best prediction is (10, 'ALL', 0.08754139647420317)
Best prediction is (12, 'ALL', 0.5937288079952544)
Best prediction is (12, 'ALL', 0.8849690490394341)
Best prediction is (12, 'ALL', 0.9125085454135978)
Best prediction is (13, 'ALL', 0.99999999998518)
Best prediction is (13, 'ALL', 0.9999997473494611)
Best prediction is (13, 'ALL', 0.4273147771128658)
Best prediction is (13, 'ALL', 0.9999999113291193)
Accuracy: 0.7608695652173914
[ 0.  0.  0.  1.  1.  2.  2.  2.  3.  3.  3.  4.  4.  4.  4.  4

,Ok,Err
Breast,1,1
Prostate,0,1
Lung,2,2
Colorectal,3,0
Lymphoma,5,0
Bladder,2,1
Melanoma,1,1
Uterus__Adeno,2,2
Leukemia,5,0
Renal,3,0


RandomState 12: Accuracy 0.7608695652173914


,Ok,Err
Breast,1,2
Prostate,0,1
Lung,2,0
Colorectal,3,0
Lymphoma,6,0
Bladder,2,1
Melanoma,1,1
Uterus__Adeno,2,1
Leukemia,5,0
Renal,3,0


...

,Ok,Err
Breast,1,1
Prostate,0,1
Lung,2,1
Colorectal,3,0
Lymphoma,5,0
Bladder,2,1
Melanoma,1,1
Uterus__Adeno,2,1
Leukemia,5,0
Renal,3,0


RandomState 9: Accuracy 0.782608695652174


,Ok,Err
Breast,1,1
Prostate,0,1
Lung,2,1
Colorectal,3,0
Lymphoma,6,0
Bladder,2,1
Melanoma,1,1
Uterus__Adeno,2,1
Leukemia,5,0
Renal,3,0


RandomState 75: Accuracy 0.7608695652173914


,Ok,Err
Breast,1,2
Prostate,0,1
Lung,2,1
Colorectal,3,0
Lymphoma,6,0
Bladder,2,0
Melanoma,1,1
Uterus__Adeno,2,2
Leukemia,5,0
Renal,3,0


In [9]:
findMaxAccuracy(1, 5, AP_SVM_Classifier.createClassifiers)



10:55:54.406281 Iteration 0: Using randomState = 37
Fitting state
Fitting classifier #[0, 1] ['Breast', 'Prostate']
Fitting classifier #[0, 2] ['Breast', 'Lung']
Fitting classifier #[0, 3] ['Breast', 'Colorectal']
Fitting classifier #[0, 4] ['Breast', 'Lymphoma']
Fitting classifier #[0, 5] ['Breast', 'Bladder']
Fitting classifier #[0, 6] ['Breast', 'Melanoma']
Fitting classifier #[0, 7] ['Breast', 'Uterus__Adeno']
Fitting classifier #[0, 8] ['Breast', 'Leukemia']
Fitting classifier #[0, 9] ['Breast', 'Renal']
Fitting classifier #[0, 10] ['Breast', 'Pancreas']
Fitting classifier #[0, 11] ['Breast', 'Ovary']
Fitting classifier #[0, 12] ['Breast', 'Mesothelioma']
Fitting classifier #[0, 13] ['Breast', 'CNS']
Fitting classifier #[1, 0] ['Prostate', 'Breast']
Fitting classifier #[1, 2] ['Prostate', 'Lung']
Fitting classifier #[1, 3] ['Prostate', 'Colorectal']
Fitting classifier #[1, 4] ['Prostate', 'Lymphoma']
Fitting classifier #[1, 5] ['Prostate', 'Bladder']
Fitting classifier #[1, 6] [

Fitting classifier #[12, 2] ['Mesothelioma', 'Lung']
Fitting classifier #[12, 3] ['Mesothelioma', 'Colorectal']
Fitting classifier #[12, 4] ['Mesothelioma', 'Lymphoma']
Fitting classifier #[12, 5] ['Mesothelioma', 'Bladder']
Fitting classifier #[12, 6] ['Mesothelioma', 'Melanoma']
Fitting classifier #[12, 7] ['Mesothelioma', 'Uterus__Adeno']
Fitting classifier #[12, 8] ['Mesothelioma', 'Leukemia']
Fitting classifier #[12, 9] ['Mesothelioma', 'Renal']
Fitting classifier #[12, 10] ['Mesothelioma', 'Pancreas']
Fitting classifier #[12, 11] ['Mesothelioma', 'Ovary']
Fitting classifier #[12, 13] ['Mesothelioma', 'CNS']
Fitting classifier #[13, 0] ['CNS', 'Breast']
Fitting classifier #[13, 1] ['CNS', 'Prostate']
Fitting classifier #[13, 2] ['CNS', 'Lung']
Fitting classifier #[13, 3] ['CNS', 'Colorectal']
Fitting classifier #[13, 4] ['CNS', 'Lymphoma']
Fitting classifier #[13, 5] ['CNS', 'Bladder']
Fitting classifier #[13, 6] ['CNS', 'Melanoma']
Fitting classifier #[13, 7] ['CNS', 'Uterus__Ade

Fitting classifier #[6, 7] ['Melanoma', 'Uterus__Adeno']
Fitting classifier #[6, 8] ['Melanoma', 'Leukemia']
Fitting classifier #[6, 9] ['Melanoma', 'Renal']
Fitting classifier #[6, 10] ['Melanoma', 'Pancreas']
Fitting classifier #[6, 11] ['Melanoma', 'Ovary']
Fitting classifier #[6, 12] ['Melanoma', 'Mesothelioma']
Fitting classifier #[6, 13] ['Melanoma', 'CNS']
Fitting classifier #[7, 0] ['Uterus__Adeno', 'Breast']
Fitting classifier #[7, 1] ['Uterus__Adeno', 'Prostate']
Fitting classifier #[7, 2] ['Uterus__Adeno', 'Lung']
Fitting classifier #[7, 3] ['Uterus__Adeno', 'Colorectal']
Fitting classifier #[7, 4] ['Uterus__Adeno', 'Lymphoma']
Fitting classifier #[7, 5] ['Uterus__Adeno', 'Bladder']
Fitting classifier #[7, 6] ['Uterus__Adeno', 'Melanoma']
Fitting classifier #[7, 8] ['Uterus__Adeno', 'Leukemia']
Fitting classifier #[7, 9] ['Uterus__Adeno', 'Renal']
Fitting classifier #[7, 10] ['Uterus__Adeno', 'Pancreas']
Fitting classifier #[7, 11] ['Uterus__Adeno', 'Ovary']
Fitting classifi

Fitting classifier #[0, 9] ['Breast', 'Renal']
Fitting classifier #[0, 10] ['Breast', 'Pancreas']
Fitting classifier #[0, 11] ['Breast', 'Ovary']
Fitting classifier #[0, 12] ['Breast', 'Mesothelioma']
Fitting classifier #[0, 13] ['Breast', 'CNS']
Fitting classifier #[1, 0] ['Prostate', 'Breast']
Fitting classifier #[1, 2] ['Prostate', 'Lung']
Fitting classifier #[1, 3] ['Prostate', 'Colorectal']
Fitting classifier #[1, 4] ['Prostate', 'Lymphoma']
Fitting classifier #[1, 5] ['Prostate', 'Bladder']
Fitting classifier #[1, 6] ['Prostate', 'Melanoma']
Fitting classifier #[1, 7] ['Prostate', 'Uterus__Adeno']
Fitting classifier #[1, 8] ['Prostate', 'Leukemia']
Fitting classifier #[1, 9] ['Prostate', 'Renal']
Fitting classifier #[1, 10] ['Prostate', 'Pancreas']
Fitting classifier #[1, 11] ['Prostate', 'Ovary']
Fitting classifier #[1, 12] ['Prostate', 'Mesothelioma']
Fitting classifier #[1, 13] ['Prostate', 'CNS']
Fitting classifier #[2, 0] ['Lung', 'Breast']
Fitting classifier #[2, 1] ['Lung'

Fitting classifier #[12, 10] ['Mesothelioma', 'Pancreas']
Fitting classifier #[12, 11] ['Mesothelioma', 'Ovary']
Fitting classifier #[12, 13] ['Mesothelioma', 'CNS']
Fitting classifier #[13, 0] ['CNS', 'Breast']
Fitting classifier #[13, 1] ['CNS', 'Prostate']
Fitting classifier #[13, 2] ['CNS', 'Lung']
Fitting classifier #[13, 3] ['CNS', 'Colorectal']
Fitting classifier #[13, 4] ['CNS', 'Lymphoma']
Fitting classifier #[13, 5] ['CNS', 'Bladder']
Fitting classifier #[13, 6] ['CNS', 'Melanoma']
Fitting classifier #[13, 7] ['CNS', 'Uterus__Adeno']
Fitting classifier #[13, 8] ['CNS', 'Leukemia']
Fitting classifier #[13, 9] ['CNS', 'Renal']
Fitting classifier #[13, 10] ['CNS', 'Pancreas']
Fitting classifier #[13, 11] ['CNS', 'Ovary']
Fitting classifier #[13, 12] ['CNS', 'Mesothelioma']
Predicting state
Best prediction is (10, 'ALL', 0.7809037004564963)
Best prediction is (0, 'ALL', 0.791452442599603)
Best prediction is (11, 'ALL', 0.813023607262294)
Best prediction is (7, 'ALL', 0.9543880009

...

,Ok,Err
Breast,1,1
Prostate,0,2
Lung,0,1
Colorectal,3,6
Lymphoma,5,0
Bladder,1,0
Melanoma,1,0
Uterus__Adeno,0,2
Leukemia,5,0
Renal,1,0


RandomState 9: Accuracy 0.5652173913043478


,Ok,Err
Breast,1,2
Prostate,0,2
Lung,0,1
Colorectal,3,6
Lymphoma,5,0
Bladder,2,0
Melanoma,1,0
Uterus__Adeno,0,2
Leukemia,5,0
Renal,1,0


RandomState 75: Accuracy 0.5869565217391305


,Ok,Err
Breast,1,3
Prostate,0,2
Lung,0,1
Colorectal,3,3
Lymphoma,5,0
Bladder,2,0
Melanoma,1,1
Uterus__Adeno,0,2
Leukemia,5,0
Renal,2,0
